# Extract Foercasted Weather Data
# -------------------------------

## Initial Imports

In [1]:
# Import Dependencies
import requests
import pandas as pd
import json
import numpy as np
import datetime
from dateutil import tz

In [2]:
# Import API Key
from config import weather_api_key

In [3]:
# Documentation
# https://openweathermap.org/api/one-call-api

# Define the Functions
# --------------------

In [4]:
def convert_Date_UTC_to_CST(UTC_date_list, list_range):
    CST_date_list = []

    for date in list_range:    
        # Convert the date/time to ISO standard in string format
        date_time = datetime.datetime.utcfromtimestamp(UTC_date_list[date]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Create a datetime object, representing the UTC time
        time_utc = datetime.datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S')

        # Replace the timezone field of the datetime object to UTC
        from_zone = tz.gettz('UTC')
        time_utc = time_utc.replace(tzinfo=from_zone)

        # Convert time zone from UTC to central
        to_zone = tz.gettz('America/Chicago')
        time_cst = time_utc.astimezone(to_zone)

        # Extract the date and store as a string
        Date = datetime.datetime.date(time_cst)
        Date = Date.strftime('%Y-%m-%d')
        CST_date_list.append(Date)
    
    return CST_date_list

In [5]:
def convert_Time_UTC_to_CST(UTC_time_list, list_range):
    CST_time_list = []

    for time in list_range:    
        # Convert the date/time to ISO standard in string format
        date_time = datetime.datetime.utcfromtimestamp(UTC_time_list[time]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Create a datetime object, representing the UTC time
        time_utc = datetime.datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S')

        # Replace the timezone field of the datetime object to UTC
        from_zone = tz.gettz('UTC')
        time_utc = time_utc.replace(tzinfo=from_zone)

        # Convert time zone from UTC to central
        to_zone = tz.gettz('America/Chicago')
        time_cst = time_utc.astimezone(to_zone)

        # Extract the Time and store as a string
        Time = time_cst.strftime('%H:%M:%S')
        CST_time_list.append(Time)
    
    return CST_time_list

In [6]:
def convert_Hour_UTC_to_CST(UTC_time_list, list_range):
    CST_hour_list = []

    for time in list_range:    
        # Convert the date/time to ISO standard in string format
        date_time = datetime.datetime.utcfromtimestamp(UTC_time_list[time]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Create a datetime object, representing the UTC time
        time_utc = datetime.datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S')

        # Replace the timezone field of the datetime object to UTC
        from_zone = tz.gettz('UTC')
        time_utc = time_utc.replace(tzinfo=from_zone)

        # Convert time zone from UTC to central
        to_zone = tz.gettz('America/Chicago')
        time_cst = time_utc.astimezone(to_zone)

        # Extract the Hour and store as an int
        Time = datetime.datetime.time(time_cst).hour
        CST_hour_list.append(Time)
    
    return CST_hour_list

# Webberville Solar Farm
# ----------------------

In [7]:
# Geographical coordinates (latitude, longitude)
lat = "30.238333"
lon = "-97.508611"

# Exclude some parts of the weather data from the API response
    # It should be a comma-delimited list (without spaces)
    # Options: current,minutely,hourly,daily,alerts
part = "minutely,alerts"

# Define the Units of measurement
    # Options: stadnard,metric,imperial
units = "imperial"

# Make a request to openweathermap
requestURL = f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude={part}&units={units}&appid={weather_api_key}"

# print(requestURL)

In [8]:
response = requests.get(requestURL)

if response.status_code == 200:
    # Turn the response into a JSON object
    responseJson = response.json()
    print("Successfully turned response into JSON object.")
else:
    # Else, print the Error status code
    print("The Error Status Code is:") 
    print(response.status_code)

Successfully turned response into JSON object.


# Current Weather Data:

## Date / Time

In [9]:
# Convert the date/time to ISO standard in string format
current_date_time = datetime.datetime.utcfromtimestamp(responseJson["current"]["dt"]).strftime('%Y-%m-%d %H:%M:%S')

# Create a datetime object, representing the UTC time
current_time_utc = datetime.datetime.strptime(current_date_time, '%Y-%m-%d %H:%M:%S')

# Replace the timezone field of the datetime object to UTC
from_zone = tz.gettz('UTC')
current_time_utc = current_time_utc.replace(tzinfo=from_zone)

# Convert time zone from UTC to central
to_zone = tz.gettz('America/Chicago')
current_time_cst = current_time_utc.astimezone(to_zone)

# Display the current time
current_time_cst

datetime.datetime(2020, 11, 1, 0, 25, 42, tzinfo=tzfile('US/Central'))

In [10]:
# Extract the date and store as a string
current_Date = datetime.datetime.date(current_time_cst)
current_Date = current_Date.strftime('%Y-%m-%d')

# Display the date
current_Date

'2020-11-01'

In [11]:
# Extract the hour from the current time
current_Hour = datetime.datetime.time(current_time_cst).hour

# Display the hour
current_Hour

0

In [12]:
# Alternative: Exract the time in %H:%M:%S
current_Time = current_time_cst.strftime('%H:%M:%S')

# Display the hour
current_Time

'00:25:42'

## Weather Description

In [13]:
# Extract the current weather description
current_Weather_Description = responseJson["current"]["weather"][0]["description"]

# Display the weather description
current_Weather_Description

'scattered clouds'

## Temperature

In [14]:
# Extract the current temperature (F)
current_Temperature_F = responseJson["current"]["temp"]

# Display the temperature
current_Temperature_F

54.55

## Sunhour

In [15]:
# Sunrise:

# Convert the date/time to ISO standard in string format
sunrise_date_time = datetime.datetime.utcfromtimestamp(responseJson["current"]["sunrise"]).strftime('%Y-%m-%d %H:%M:%S')

# Create a datetime object, representing the UTC time
sunrise_utc = datetime.datetime.strptime(sunrise_date_time, '%Y-%m-%d %H:%M:%S')

# Replace the timezone field of the datetime object to UTC
from_zone = tz.gettz('UTC')
sunrise_utc = sunrise_utc.replace(tzinfo=from_zone)

# Convert time zone from UTC to central
to_zone = tz.gettz('America/Chicago')
todays_sunrise = sunrise_utc.astimezone(to_zone)

# Display sunrise time
todays_sunrise

datetime.datetime(2020, 11, 1, 6, 44, 37, tzinfo=tzfile('US/Central'))

In [16]:
# Sunset:

# Convert the date/time to ISO standard in string format
sunset_date_time = datetime.datetime.utcfromtimestamp(responseJson["current"]["sunset"]).strftime('%Y-%m-%d %H:%M:%S')

# Create a datetime object, representing the UTC time
sunset_utc = datetime.datetime.strptime(sunset_date_time, '%Y-%m-%d %H:%M:%S')

# Replace the timezone field of the datetime object to UTC
from_zone = tz.gettz('UTC')
sunset_utc = sunset_utc.replace(tzinfo=from_zone)

# Convert time zone from UTC to central
to_zone = tz.gettz('America/Chicago')
todays_sunset = sunset_utc.astimezone(to_zone)

# Display sunset time
todays_sunset

datetime.datetime(2020, 11, 1, 17, 42, 29, tzinfo=tzfile('US/Central'))

## Cloud Cover

In [17]:
# Extract the current cloud cover (%)
current_CloudCover_percent = responseJson["current"]["clouds"]

# Display the Cloudiness, %
current_CloudCover_percent

40

## uv Index

In [18]:
# Extract the current UV Index
todays_uvIndex = responseJson["current"]["uvi"]

# Display the Midday UV index
todays_uvIndex

5.49

## Humidity

In [19]:
# Extract the current humidity (%)
current_Humidity_percent = responseJson["current"]["humidity"]

# Display the Cloudiness, %
current_Humidity_percent

87

# Daily Forecasted Weather Data:

In [20]:
# Get the current year, month, and day
year = current_time_cst.year
month = current_time_cst.month
day = current_time_cst.day

# Store the next day as a variable
next_day = datetime.date(year, month, day) + datetime.timedelta(days=1)
next_day

datetime.date(2020, 11, 2)

In [21]:
forecasted_daily_weather = []

for day in np.arange(0, 8, 1):
    forecasted_daily_weather.append({
        "UTC_Time": responseJson["daily"][day]["dt"],
        "Sunrise": responseJson["daily"][day]["sunrise"],
        "Sunset": responseJson["daily"][day]["sunset"],
        "uvIndex": responseJson["daily"][day]["uvi"]
    })

daily_weather_DF = pd.DataFrame(forecasted_daily_weather)
daily_weather_DF

,UTC_Time,Sunrise,Sunset,uvIndex
0,1604253600,1604234677,1604274149,5.49
1,1604340000,1604321125,1604360502,5.12
2,1604426400,1604407573,1604446856,5.19
3,1604512800,1604494021,1604533211,5.05
4,1604599200,1604580469,1604619568,4.74
5,1604685600,1604666918,1604705926,4.90
6,1604772000,1604753368,1604792286,4.97
7,1604858400,1604839817,1604878647,5.20


In [22]:
daily_index = np.arange(0,8,1)

daily_UTC_date_time = daily_weather_DF["UTC_Time"]
UTC_sunrise = daily_weather_DF["Sunrise"]
UTC_sunset = daily_weather_DF["Sunset"]

daily_forecast_dates = convert_Date_UTC_to_CST(daily_UTC_date_time, daily_index)
daily_sunrise_times = convert_Time_UTC_to_CST(UTC_sunrise, daily_index)
daily_sunset_times = convert_Time_UTC_to_CST(UTC_sunset, daily_index)

In [23]:
# Add the date to the dataframe
daily_weather_DF["Date"] = daily_forecast_dates
# Add the date to the dataframe
daily_weather_DF["Sunrise"] = daily_sunrise_times
# Add the date to the dataframe
daily_weather_DF["Sunset"] = daily_sunset_times
# Drop the UTC_Time column
daily_weather_DF.drop(columns=["UTC_Time"], axis=1, inplace=True)
# Re-Order the columns
daily_weather_DF = daily_weather_DF[["Date", "Sunrise", "Sunset", "uvIndex"]]

daily_weather_DF

,Date,Sunrise,Sunset,uvIndex
0,2020-11-01,06:44:37,17:42:29,5.49
1,2020-11-02,06:45:25,17:41:42,5.12
2,2020-11-03,06:46:13,17:40:56,5.19
3,2020-11-04,06:47:01,17:40:11,5.05
4,2020-11-05,06:47:49,17:39:28,4.74
5,2020-11-06,06:48:38,17:38:46,4.90
6,2020-11-07,06:49:28,17:38:06,4.97
7,2020-11-08,06:50:17,17:37:27,5.20


In [24]:
# Check the data types
daily_weather_DF.dtypes

Date        object
Sunrise     object
Sunset      object
uvIndex    float64
dtype: object

# Hourly Forecasted Weather Data:

In [25]:
forecasted_hourly_weather = []

for hour in np.arange(0, 48, 1):
    forecasted_hourly_weather.append({
        "UTC_Time": responseJson["hourly"][hour]["dt"],
        "Weather_Description": responseJson["hourly"][hour]["weather"][0]["description"],
        "CloudCover_percent": responseJson["hourly"][hour]["clouds"],
        "Humidity_percent": responseJson["hourly"][hour]["humidity"]
    })

hourly_weather_DF = pd.DataFrame(forecasted_hourly_weather)
hourly_weather_DF.head()

,UTC_Time,Weather_Description,CloudCover_percent,Humidity_percent
0,1604206800,scattered clouds,40,87
1,1604210400,broken clouds,58,69
2,1604214000,overcast clouds,87,61
3,1604217600,overcast clouds,96,57
4,1604221200,overcast clouds,100,55


In [26]:
hourly_index = np.arange(0,48,1)

hourly_UTC_date_time = hourly_weather_DF["UTC_Time"]

hourly_forecast_dates = convert_Date_UTC_to_CST(hourly_UTC_date_time, hourly_index)
hourly_forecast_times = convert_Hour_UTC_to_CST(hourly_UTC_date_time, hourly_index)

In [27]:
# Add the date to the dataframe
hourly_weather_DF["Date"] = hourly_forecast_dates
# Add the date to the dataframe
hourly_weather_DF["Time"] = hourly_forecast_times
# Drop the UTC_Time column
hourly_weather_DF.drop(columns=["UTC_Time"], axis=1, inplace=True)
# Re-Order the columns
hourly_weather_DF = hourly_weather_DF[["Date", "Time", "Weather_Description", "CloudCover_percent", "Humidity_percent"]]

hourly_weather_DF.head()

,Date,Time,Weather_Description,CloudCover_percent,Humidity_percent
0,2020-11-01,0,scattered clouds,40,87
1,2020-11-01,1,broken clouds,58,69
2,2020-11-01,1,overcast clouds,87,61
3,2020-11-01,2,overcast clouds,96,57
4,2020-11-01,3,overcast clouds,100,55


In [28]:
# Check the data types
hourly_weather_DF.dtypes

Date                   object
Time                    int64
Weather_Description    object
CloudCover_percent      int64
Humidity_percent        int64
dtype: object

# Merge the Daily and Hourly Forecasted Weather DataFrames:

In [29]:
forecasted_weather_DF = pd.merge(hourly_weather_DF, daily_weather_DF, on='Date', how='outer')
forecasted_weather_DF.dropna(inplace=True)
forecasted_weather_DF['Date'] = pd.to_datetime(forecasted_weather_DF['Date'])
forecasted_weather_DF["Time"] = forecasted_weather_DF["Time"].astype(int)

forecasted_weather_DF.head()

,Date,Time,Weather_Description,CloudCover_percent,Humidity_percent,Sunrise,Sunset,uvIndex
0,2020-11-01,0,scattered clouds,40.0,87.0,06:44:37,17:42:29,5.49
1,2020-11-01,1,broken clouds,58.0,69.0,06:44:37,17:42:29,5.49
2,2020-11-01,1,overcast clouds,87.0,61.0,06:44:37,17:42:29,5.49
3,2020-11-01,2,overcast clouds,96.0,57.0,06:44:37,17:42:29,5.49
4,2020-11-01,3,overcast clouds,100.0,55.0,06:44:37,17:42:29,5.49


In [30]:
forecasted_weather_DF.dtypes

Date                   datetime64[ns]
Time                            int32
Weather_Description            object
CloudCover_percent            float64
Humidity_percent              float64
Sunrise                        object
Sunset                         object
uvIndex                       float64
dtype: object

# Hackberry Wind Farm
# -------------------

# Weather Descriptions
# --------------------

In [31]:
weather_description_list = [
    # Clear
    'Clear',
    'Sunny',
    # Clouds
    'Partly cloudy', 
    'Cloudy', 
    'Overcast',
    # Drizzle
    'Freezing drizzle',
    'Light drizzle', 
    'Patchy light drizzle',
    # Snow
    'Patchy moderate snow', 
    'Moderate snow', 
    'Light sleet',
    'Light snow',  
    'Heavy snow', 
    'Patchy heavy snow', 
    'Blowing snow',  
    'Blizzard',
    'Ice pellets', 
    'Patchy light snow', 
    # Rain
    'Patchy rain possible',
    'Light rain shower', 
    'Moderate or heavy rain shower',
    'Moderate rain', 
    'Heavy rain', 
    'Light rain', 
    'Patchy light rain', 
    'Torrential rain shower',
    'Moderate rain at times', 
    'Heavy rain at times', 
    'Moderate or heavy freezing rain',
    # Thunderstorm
    'Thundery outbreaks possible',
    'Moderate or heavy rain with thunder',
    'Patchy light rain with thunder', 
    # Atmosphere
    'Mist',
    'Fog']

In [32]:
weather_condition_codes = [
    # Thunderstorm
    'thunderstorm with light rain',
    'thunderstorm with rain',
    'thunderstorm with heavy rain',
    'light thunderstorm',
    'thunderstorm',
    'heavy thunderstorm',
    'ragged thunderstorm',
    'thunderstorm with light drizzle',
    'thunderstorm with drizzle',
    'thunderstorm with heavy drizzle',
    # Drizzle
    'light intensity drizzle',
    'drizzle',
    'heavy intensity drizzle',
    'light intensity drizzle rain',
    'drizzle rain',
    'heavy intensity drizzle rain',
    'shower rain and drizzle',
    'heavy shower rain and drizzle',
    'shower drizzle',
    # Rain
    'light rain',
    'moderate rain',
    'heavy intensity rain',
    'very heavy rain',
    'extreme rain',
    'freezing rain',
    'light intensity shower rain',
    'shower rain',
    'heavy intensity shower rain',
    'ragged shower rain',
    # Snow
    'light snow',
    'Snow',
    'Heavy snow',
    'Sleet',
    'Light shower sleet',
    'Shower sleet',
    'Light rain and snow',
    'Rain and snow',
    'Light shower snow',
    'Shower snow',
    'Heavy shower snow',
    # Atmosphere
    'mist',
    'Smoke',
    'Haze',
    'sand/ dust whirls',
    'fog',
    'sand',
    'dust',
    'volcanic ash',
    'squalls',
    'tornado',
    # Clear
    'clear sky',
    # Clouds
    'few clouds',
    'scattered clouds',
    'broken clouds',
    'overcast clouds'   
]

In [33]:
weather_icons = [
    "clear sky",
    "few clouds",
    "scattered clouds",
    "broken clouds",
    "shower rain",
    "rain",
    "thunderstorm",
    "snow",
    "mist"
]